话说实例30中填写的表格太丑了（下图），老板很生气，后果很严重。这次我们就来修饰美化一下。
![](images\result-actual.png)
有几个要求：
1. 最后一行中，“总数”两个字要加粗
2. 最后一行的行高要调大一点，现在这样扁扁的，难看
3. 最后一行那几个空单元格有框线，太碍眼，得去掉
4. 各单元格水平和垂直方向都要居中对齐
5. 数量列≥85的，设置颜色填充

搞清楚要求之后，就可以开工了。由于`docx`模块没有比较友好的函数可直接调用，如下边框处理函数是从其他大神那里copy来的，我自己也看不大明白，只要会使用，能解决问题就成，这也是我们学习编程的初衷。有轮子就用，不必认死扣，非要成为轮子制造专家。我在如下函数中增加了使用说明，你也可以直接复制如下函数使用。其中，传入参数有cell, 即单元格；top指上边框；bottom指下边框；start指左边框；end指右边框。"sz"指线的粗细程度（取值在2到96之间）；"val"指线型，比如单线，虚线等；"color"指颜色。"space"指间隔，一般不设置，设置的值大于0会导致线错开；"shadow"指边框阴影。下图是space为0和10的对比。
![](images\space.png)

In [26]:
#单元格边框设置函数
from docx.table import _Cell
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
 
def Set_cell_border(cell: _Cell, **kwargs):
    """
    设置单元格边框函数
    使用方法:
    Set_cell_border(
        cell,
        top={"sz": 12, "val": "single", "color": "#FF0000", "space": "0"},
        bottom={"sz": 12, "color": "#00FF00", "val": "single"},
        start={"sz": 24, "val": "dashed", "shadow": "true"},
        end={"sz": 12, "val": "dashed"},
    )
    传入参数有cell, 即单元格；top指上边框；bottom指下边框；start指左边框；end指右边框。
    "sz"指线的粗细程度；"val"指线型，比如单线，虚线等；"color"指颜色，颜色编码可百度；
    "space"指间隔，一般不设置，设置的值大于0会导致线错开；"shadow"指边框阴影
    """
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()
    
    tcBorders = tcPr.first_child_found_in("w:tcBorders")
    if tcBorders is None:
        tcBorders = OxmlElement('w:tcBorders')
        tcPr.append(tcBorders)
    
    for edge in ('start', 'top', 'end', 'bottom', 'insideH', 'insideV'):
        edge_data = kwargs.get(edge)
        if edge_data:
            tag = 'w:{}'.format(edge)
            
            element = tcBorders.find(qn(tag))
            if element is None:
                element = OxmlElement(tag)
                tcBorders.append(element)
 
            for key in ["sz", "val", "color", "space", "shadow"]:
                if key in edge_data:
                    element.set(qn('w:{}'.format(key)), str(edge_data[key]))

除了边框设置，还要定义一个单元格颜色填充函数，以便调用，给满足条件的单元格填充颜色，让表格看起来更美观。下面的填色函数比较简单，只需往里面传入`cell`即单元格，和RGB颜色编码即可。RGB颜色编码可在网上查询（网址： https://www.114la.com/other/rgb.htm ）。比如红色的编码为“FF0000”。

In [27]:
#定义单元格填充颜色函数
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
def Set_Background_Color(cell,rgbColor):
    shading_elm = parse_xml(r'<w:shd {} w:fill="{color_value}"/>'.format(nsdecls('w'),color_value = rgbColor)) #固定写法，照抄即可
    cell._tc.get_or_add_tcPr().append(shading_elm)

准备好以上函数后，就可以开始主程序了。先导入对齐和厘米长度单位所要用到的库。然后打开需要处理的文件"收货记录"，锁定第一个表格`doc.tables[0]`，并存入`table`变量，以便后面使用。通过`len(table.rows)`获取表格总行数，存入变量`max_row`，以便后续做操作的时候精准定位。然后就可以开始第一步操作了，即给最后一行的"总数"二字加粗。由于只有文字块才有字体`font`属性，所以我们需要抓取到总数所在单元格的文字块，即`run`。这里有个层级关系，即“单元格”(`cell`)往下是“段落”(`paragraphs`)，再往下是“文字块”(`runs`)。我们已知这个单元格中只有“总数：”这几个字符，所以这个单元格里只有一个段落，一个文字块，通过`.paragraphs[0].runs[0]`直接锁定第一个段落的第一个文字块。然后通过`run.font.bold = True`将其设置为粗体。

然后开始第二项要求的操作。这个操作很简单，直接将最后一行的行高设置为1厘米。`Cm() `括号内填入数字，即代表多少厘米，比如Cm(2.2)表示2.2厘米。

第三项要求也算是比较简单的，直接调用上面定义好的边框设置函数即可。此处，我们只想把没有内容的边框去掉，直接将其设置为白色即可，即`"color": "#FFFFFF"`。对于最后一行前三个单元格，需要将左、右和下边框设置为白色，这里用了一个`for`循环来做这个重复操作。对于第四个单元格，只需要将左和下边框进行设置，所以只传入`start`和`bottom`两个边框的颜色。对于最后一个单元格，只需要将右和下边框进行设置，所以只传入`end`和`bottom`两个边框的颜色。

第四项要求是所有单元格水平即垂直居中对齐。直接`for`循环遍历所有单元格，然后进行对齐设置即可。对于水平对齐，需要去到“段落” (`paragraphs`)才行，由于每个单元格只有一段，所以通过`.paragraphs[0]`直接锁定。垂直对齐就简单一点，直接`.vertical_alignment`，并指定为`WD_ALIGN_VERTICAL.CENTER`即可，其中`.CENTER`表示居中。

第五项要求是数量列≥85的单元格，设置颜色填充。在设置之前，需要先获取数量列的所有数据，然后用`if`语句判断，如果其值≥85，才调用填色函数填色。所以先建一个列表`qty`，从表格中将所有数取出，并转换为整数型`int`存入。取出的值会默认为字符串，不转成整数，将无法与85来比较。在比较之前，先定义一个行计数器`row = 1`，起始值为1（因为我们是从第二行开始的，其行索引为1）,每比较一行，就加1，以便跳转到下一行。每比较一行，如果满足条件，就将该行对应的单元格通过调用填色函数`Set_Background_Color`填充颜色。

In [28]:
from docx.enum.text import WD_ALIGN_PARAGRAPH #设置水平居中对齐需要用到的库
from docx.enum.table import WD_ALIGN_VERTICAL #设置垂直居中对齐需要用到的库
from docx.shared import Cm # 长度单位（厘米cm用到的库)
from docx import Document

doc = Document("收货记录.docx") #打开word文件
table= doc.tables[0]
max_row = len(table.rows) #获取表格总行数

#1.最后一行中，“总数”两个字要加粗
run = table.cell(max_row-1,4).paragraphs[0].runs[0] # 获取最后一行第五列对应单元格中的文字块，即“总数”字样所在单元格
run.font.bold = True #将文字块设置为粗体

#2.最后一行的行高要调大一点，现在这样扁扁的，难看
table.rows[max_row-1].height = Cm(1) # 将最后一行的行高设置为1厘米

#3.最后一行那几个空单元格有框线，太碍眼，得去掉，将总数及左边两个cell左边框和下边框设成白色
for i in range(3):
    cell = table.cell(max_row-1,i)
    Set_cell_border(
            cell,
            bottom={"color": "#FFFFFF"},
            start={"color": "#FFFFFF" },
            end={"color": "#FFFFFF"}
        )

cell_1 = table.cell(max_row-1,3)
Set_cell_border(cell_1,start={"color": "#FFFFFF" },bottom={"color": "#FFFFFF"})

cell_2 = table.cell(max_row-1,6)
Set_cell_border(cell_2,end={"color": "#FFFFFF" },bottom={"color": "#FFFFFF"})

#4.各单元格水平和垂直方向都要居中对齐
for row in range(1,max_row):
    for col in range(len(table.columns)):
        table.cell(row,col).paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
        table.cell(row,col).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        
#5.数量列≥85的，底色设置为橙色
qty = [] #存储数量信息
#读取第二行到29行，第2，3列中的数据
for i in range(1,max_row-1):
    qty_info = table.rows[i].cells[5].text #cells[5]指表格第6列
    qty.append(int(qty_info))
    
#将数量≥85的单元格填色
row=1 #行计数器
for i in qty:
    if i>=85:
        cell = table.cell(row,5) #第6列数据为数量，列索引是5
        Set_Background_Color(cell,"98F5FF") #填充颜色，"98F5FF"是蓝色的编码
    row+=1 #跳转到下一行      

doc.save("收货记录-整理.docx")

最后结果如下，是不是还阔以啊？
![](images\result.png)